In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [22]:
import mysql.connector
from collections import defaultdict

# Database connection
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get payouts
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Quarterfinals', 'Semifinals Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'quarterfinals_payout': 0, 'semifinal_payout': 0, 'championship_payout': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == "quarterfinals":
        payouts[normalized_name]['quarterfinals_payout'] = float(total_payout)
    elif event_type.strip().lower() == "semifinals winner":
        payouts[normalized_name]['semifinal_payout'] = float(total_payout)
    elif event_type.strip().lower() == "championship":
        payouts[normalized_name]['championship_payout'] = float(total_payout)

cursor.close()
conn.close()

# **Teams and Correct Quarterfinal Win Probabilities**
teams = {
    "kansas city chiefs": {"current_round_prob": 0.7},
    "houston texans": {"current_round_prob": 0.3},
    "buffalo bills": {"current_round_prob": 0.6},
    "baltimore ravens": {"current_round_prob": 0.4},
    "detroit lions": {"current_round_prob": 0.55},
    "washington commanders": {"current_round_prob": 0.45},
    "philadelphia eagles": {"current_round_prob": 0.52},
    "los angeles rams": {"current_round_prob": 0.48}
}

# **Hypothetical Probabilities for Semifinals & Beyond**
hypothetical_probabilities = {
    "kansas city chiefs": {"houston texans": 0.5, "buffalo bills": 0.55, "baltimore ravens": 0.65, "detroit lions": 0.5, "philadelphia eagles": 0.55, "los angeles rams": 0.52, "washington commanders": 0.5},
    "houston texans": {"kansas city chiefs": 0.5, "buffalo bills": 0.6, "baltimore ravens": 0.45, "detroit lions": 0.5, "philadelphia eagles": 0.52, "los angeles rams": 0.48, "washington commanders": 0.55},
    "buffalo bills": {"kansas city chiefs": 0.45, "baltimore ravens": 0.5, "houston texans": 0.4, "detroit lions": 0.55, "philadelphia eagles": 0.5, "los angeles rams": 0.53, "washington commanders": 0.42},
    "baltimore ravens": {"kansas city chiefs": 0.35, "buffalo bills": 0.5, "houston texans": 0.55, "detroit lions": 0.55, "philadelphia eagles": 0.5, "los angeles rams": 0.55, "washington commanders": 0.48},
    "detroit lions": {"washington commanders": 0.5, "philadelphia eagles": 0.6, "los angeles rams": 0.51, "kansas city chiefs": 0.5, "buffalo bills": 0.45, "baltimore ravens": 0.45, "houston texans": 0.48},
    "washington commanders": {"detroit lions": 0.5, "philadelphia eagles": 0.45, "los angeles rams": 0.5, "kansas city chiefs": 0.5, "buffalo bills": 0.4, "baltimore ravens": 0.5, "houston texans": 0.45},
    "philadelphia eagles": {"washington commanders": 0.55, "los angeles rams": 0.45, "detroit lions": 0.4, "kansas city chiefs": 0.55, "buffalo bills": 0.5, "baltimore ravens": 0.5, "houston texans": 0.52},
    "los angeles rams": {"philadelphia eagles": 0.55, "washington commanders": 0.5, "detroit lions": 0.5, "kansas city chiefs": 0.48, "buffalo bills": 0.53, "baltimore ravens": 0.55, "houston texans": 0.5}
}

# **Calculate Championship EV**
def calculate_championship_ev(winner, same_conference, other_conference):
    championship_ev = 0
    for x in same_conference:
        if x != winner:
            P_sf_win_tx = hypothetical_probabilities[winner][x]
            P_sf_win_xt = hypothetical_probabilities[x][winner]
            P_qf_win_x = teams[x]["current_round_prob"]

            for y in same_conference:
                P_conf_win_y = (
                    teams[y]["current_round_prob"]
                    * hypothetical_probabilities[y][winner if y != winner else x]
                )
                for z in other_conference:
                    P_conf_win_z = (
                        teams[z]["current_round_prob"]
                        * sum(
                            teams[y_prime]["current_round_prob"]
                            * hypothetical_probabilities[y_prime][z]
                            for y_prime in same_conference if y_prime != z
                        )
                    )
                    P_champ_win_yz = hypothetical_probabilities[y][z]
                    P_champ_win_zy = hypothetical_probabilities[z][y]
                    P_champ_payout_y = payouts[y]["championship_payout"]
                    P_champ_payout_z = payouts[z]["championship_payout"]

                    championship_ev += (
                        P_sf_win_tx * P_qf_win_x * P_conf_win_y
                        * (P_champ_win_yz * P_conf_win_z * P_champ_payout_y
                        + P_champ_win_zy * P_conf_win_z * P_champ_payout_z)
                    )
    return championship_ev

# **Calculate Portfolio EV**
def calculate_portfolio_ev(winner, same_conference, other_conference):
    # Quarterfinal EV
    quarterfinals_ev = teams[winner]["current_round_prob"] * payouts[winner]["quarterfinals_payout"]
    for x in same_conference:
        if x != winner:
            quarterfinals_ev += teams[x]["current_round_prob"] * hypothetical_probabilities[winner][x] * payouts[x]["quarterfinals_payout"]

    # This Conference EV
    this_conference_ev = 0
    for x in same_conference:
        if x != winner:
            P_qf_win_x = teams[x]["current_round_prob"]
            P_sf_win_tx = hypothetical_probabilities[winner][x]
            P_sf_win_xt = hypothetical_probabilities[x][winner]
            this_conference_ev += (
                P_qf_win_x
                * (P_sf_win_tx * payouts[winner]["semifinal_payout"]
                   + P_sf_win_xt * payouts[x]["semifinal_payout"])
            )

    # Other Conference EV
    other_conference_ev = 0
    for x in other_conference:
        P_qf_win_x = teams[x]["current_round_prob"]
        P_conf_win_x = P_qf_win_x * sum(
            teams[y]["current_round_prob"] * hypothetical_probabilities[y][x]
            for y in other_conference if y != x
        )
        other_conference_ev += P_conf_win_x * payouts[x]["semifinal_payout"]

    # Championship EV
    championship_ev = calculate_championship_ev(winner, same_conference, other_conference)

    return quarterfinals_ev + this_conference_ev + other_conference_ev + championship_ev

# **Define Conference Brackets**
conference_a = ["kansas city chiefs", "houston texans", "buffalo bills", "baltimore ravens"]
conference_b = ["detroit lions", "washington commanders", "philadelphia eagles", "los angeles rams"]

# **Compute Portfolio EVs for Each Scenario**
ev_results = {}
for winner in teams:
    same_conference = conference_a if winner in conference_a else conference_b
    other_conference = conference_b if winner in conference_a else conference_a

    ev = calculate_portfolio_ev(winner, same_conference, other_conference)
    ev_results[f"{winner.title()} wins quarterfinal"] = ev

# **Print Results**
print("\n==== Portfolio EVs ====")
for outcome, ev in ev_results.items():
    print(f"Portfolio EV if {outcome}: ${ev:,.2f}")



==== Portfolio EVs ====
Portfolio EV if Kansas City Chiefs wins quarterfinal: $139,393.73
Portfolio EV if Houston Texans wins quarterfinal: $150,218.95
Portfolio EV if Buffalo Bills wins quarterfinal: $117,254.53
Portfolio EV if Baltimore Ravens wins quarterfinal: $150,776.17
Portfolio EV if Detroit Lions wins quarterfinal: $121,138.33
Portfolio EV if Washington Commanders wins quarterfinal: $129,503.20
Portfolio EV if Philadelphia Eagles wins quarterfinal: $115,762.90
Portfolio EV if Los Angeles Rams wins quarterfinal: $126,605.53
